<a href="https://colab.research.google.com/github/BarryLiu-97/Tensorflow-Tutorial/blob/master/basic/tf_exp_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自动求导机制
GradientTape类 记录梯度数据的磁带
GradientTape(persistent, watch_accessed_variables)    
perisistent=False 该tape只求导一次，求导之后就被销毁  
perisistent=True 可以多次求导  
watch_accessed_variables 自动监视所有的可训练变量 取值默认为True  
```  
with GradientTape() as tape:__   
  函数表达式 
grad=tape.gradient(函数，自变量)   
```   


In [0]:
import tensorflow as tf
import numpy as np

In [0]:
x = tf.Variable(3.)
with tf.GradientTape() as tape:
  y = tf.square(x)
dy_dx = tape.gradient(y,x)

In [4]:
print(y)
print(dy_dx)

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


In [5]:
x = tf.Variable(3.)
with tf.GradientTape(persistent=True) as tape:
  y = tf.square(x)
  z = pow(x,3)  #z等于x的三次方
dy_dx = tape.gradient(y,x)
dz_dx = tape.gradient(z,x)

print(y)
print(dy_dx)
print(z)
print(dz_dx)

del tape  #persistent=True时需使用del对tape进行释放

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(27.0, shape=(), dtype=float32)
tf.Tensor(27.0, shape=(), dtype=float32)


In [8]:
x = tf.Variable(3.)
with tf.GradientTape(watch_accessed_variables=False) as tape:
  y = tf.square(x)
dy_dx_1 = tape.gradient(y,x)
print(dy_dx_1)

None


添加监视 watch(变量)  
监视非可训练变量  

In [10]:
x = tf.Variable(3.)
with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(x)   #通过手动添加watch对变量的监视
  y = tf.square(x)
dy_dx_1 = tape.gradient(y,x)
print(dy_dx_1)

tf.Tensor(6.0, shape=(), dtype=float32)


In [11]:
x = tf.constant(3.)  #tf.constant()不是一个可训练变量
with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(x)   #通过手动添加watch对变量的监视
  y = tf.square(x)
dy_dx_1 = tape.gradient(y,x)
print(dy_dx_1)

tf.Tensor(6.0, shape=(), dtype=float32)


## 多元函数求偏导数
tape.graident(函数，自变量)  
其中自变量可以是一个，也可以是多个  


In [24]:
x = tf.Variable(3.)
y = tf.Variable(4.)

with tf.GradientTape() as tape:
  f = tf.square(x) + 2*tf.square(y) + 1

df_dx, df_dy = tape.gradient(f,[x,y])  #也可以赋值给一个变量，返回一个列表
print(df_dx)
print(df_dy)

tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(16.0, shape=(), dtype=float32)


## 求二阶导数

In [26]:
x = tf.Variable(3.)
y = tf.Variable(4.)

with tf.GradientTape(persistent=True) as tape2:
  with tf.GradientTape(persistent=True) as tape1:
    f = tf.square(x) + 2*tf.square(y) + 1
  
  first_grads = tape1.gradient(f,[x,y])
second_grads = [tape2.gradient(first_grads,[x,y])]  #注意，这里因为firat_grads是一个列表，所以需要添加中括号
print(f)
print(first_grads)
print(second_grads)

del tape1
del tape2

tf.Tensor(42.0, shape=(), dtype=float32)
[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, <tf.Tensor: shape=(), dtype=float32, numpy=16.0>]
[[<tf.Tensor: shape=(), dtype=float32, numpy=2.0>, <tf.Tensor: shape=(), dtype=float32, numpy=4.0>]]


## 对向量/矩阵求导

In [27]:
x = tf.Variable([1.,2.,3.])
y = tf.Variable([4.,5.,6.])

with tf.GradientTape() as tape:
  f = tf.square(x) + 2*tf.square(y) + 1

df_dx, df_dy = tape.gradient(f,[x,y])  #也可以赋值给一个变量，返回一个列表
print(f)
print(df_dx)
print(df_dy)

tf.Tensor([34. 55. 82.], shape=(3,), dtype=float32)
tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32)
tf.Tensor([16. 20. 24.], shape=(3,), dtype=float32)
